In [ ]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 
!pip install pythainlp
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 4.8 MB/s 
     |████████████████████████████████| 1.2 MB 39.4 MB/s 
     |████████████████████████████████| 880 kB 59.4 MB/s 
     |████████████████████████████████| 101 kB 10.3 MB/s 
     |████████████████████████████████| 596 kB 66.5 MB/s 
     |████████████████████████████████| 3.3 MB 39.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ada9aea4e993b4680a450b85029473d96c75eda6c5f3cc8b91769e72711b57d0
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███

In [ ]:
from transformers import AutoModelForMaskedLM, pipeline
from transformers import AutoTokenizer, BertForTokenClassification, AutoModel
import pandas as pd
import torch
import pickle
from tqdm import tqdm
from datasets import load_metric
from pythainlp.benchmarks import word_tokenization
import numpy as np
from nltk.translate.gleu_score import sentence_gleu
from jiwer import cer

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', model_max_length=512)
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/json/tokenizer_json_15k.pkl', 'rb'))

In [ ]:
class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output

In [ ]:
tagging_model = BertModel()
# model = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
# model.resize_token_embeddings(len(tokenizer_th))

# FILE = "drive/MyDrive/AIBuilders/tagging.pth"
FILE = "drive/MyDrive/AIBuilders/tagging_json_200.pth"
tagging_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
tagging_model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.10.attention.self.key.bias', 'roberta.encoder.layer.9.attention.output.dense.bias', 'roberta.encoder.layer.2.attention.self.query.bias', 'roberta.encoder.layer.9.output.dense.weight', 'roberta.encoder.layer.11.attention.output.LayerNorm.bias', 'roberta.encoder.layer.9.attention.output.dense.weight', 'roberta.encoder.layer.0.output.dense.weight', 'roberta.encoder.layer.3.output.dense.bias', 'roberta.encoder.layer.11.output.LayerNorm.bias', 'roberta.encoder.layer.7.output.dense.bias', 'roberta.encoder.layer.11.output.LayerNorm.weight', 'roberta.encoder.layer.11.output.dense.weight', 'roberta.encoder.layer.7.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(25354, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [ ]:
ids_to_labels = {0: 'f', 1: 'i'}

def align_word_ids(texts):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)
    c = tokenizer.convert_ids_to_tokens(tokenized_inputs.input_ids)
    print(f"ids_to_tokens: {c}")
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        else:
            try:
              label_ids.append(2)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx
    return label_ids

def evaluate_one_text(model, sentence):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
    mask = text['attention_mask'][0].unsqueeze(0).to(device)
    input_id = text['input_ids'][0].unsqueeze(0).to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    # input_id = torch.Tensor([sentence]).type(torch.int64)
    # label_ids = []
    # for i in sentence:
    #   if i == 1 or i == 5 or i == 6:
    #     label_ids.append(-100)
    #   else:
    #     label_ids.append(2)
    # label_ids = torch.Tensor([label_ids]).type(torch.int64)
    # mask = torch.Tensor([mask]).type(torch.int64)

    logits = tagging_model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
mlm_model = AutoModelForMaskedLM.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
mlm_model.resize_token_embeddings(len(tokenizer))

FILE = "drive/MyDrive/AIBuilders/mlm_json_6.pth"
mlm_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
mlm_model = mlm_model.to(device)
mlm_model.eval()

CamembertForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(25354, 768)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [ ]:
ds = pickle.load(open('drive/MyDrive/AIBuilders/json/mlm_json_15k.pkl', 'rb'))
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/json/ner_json_15k.pkl', 'rb'))

In [ ]:
ds

,input_ids,attention_mask,labels
0,"[5, 10, 25004, 25004, 25004, 1147, 118, 19105,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[5, 10, 31, 265, 265, 1147, 118, 19105, 3166, ..."
1,"[5, 2169, 25088, 10, 10022, 10, 234, 112, 234,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 2169, 25088, 10, 10022, 10, 234, 112, 234,..."
2,"[5, 10, 3919, 77, 10, 25004, 25004, 25004, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 3919, 77, 10, 25005, 265, 265, 10, 825..."
3,"[5, 10, 16369, 12, 2105, 19319, 10, 300, 25004...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 16369, 12, 2105, 19319, 10, 300, 70, 1..."
4,"[5, 10, 2638, 22683, 3927, 716, 1491, 2552, 16...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[5, 10, 2638, 22683, 3927, 716, 1491, 2552, 16..."
...,...,...,...
15571,"[5, 10, 3919, 1983, 1466, 803, 7177, 75, 25004...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[5, 10, 3919, 1983, 1466, 803, 7177, 75, 25040..."
15572,"[5, 1896, 25004, 25004, 25004, 25004, 6, 1, 1,...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 1896, 1466, 265, 415, 73, 6, 1, 1, 1, 1, 1..."
15573,"[5, 2169, 25004, 905, 25098, 10, 25004, 25004,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[5, 2169, 12355, 905, 25098, 10, 415, 260, 6, ..."
15574,"[5, 10, 112, 78, 10, 25004, 25004, 73, 6, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[5, 10, 112, 78, 10, 25117, 265, 73, 6, 1, 1, ..."


In [ ]:
seq_ds = pickle.load(open('drive/MyDrive/AIBuilders/json/seq_json_15k.pkl', 'rb'))

In [ ]:
seq_ds

,text,labels
0,หริอมีปัญหาอะไรช่วยตอบด้วยครับ,หรือมีปัญหาอะไรช่วยตอบด้วยครับ
1,อยากสมัคร sms เงินเข้าเงินออกทางเน็ตต้องทําย...,อยากสมัคร sms เงินเข้าเงินออกทางเน็ตต้องทําย...
2,ไม่ทราบว่าคุณ พัชชา า ไม่สนโลก เปลี่ยนเฉพาะเค...,ไม่ทราบว่าคุณ พัชชา ไม่สนโลก เปลี่ยนเฉพาะเครื...
3,ทํารายการที่ตู้atm อะคะ เป็นการทํารายการเปลี่...,ทํารายการที่ตู้atm อะค่ะ เป็นการทํารายการเปลี...
4,ย่ายังดีที่สิริตอบแบบนั้น ดูของเค้าดิ,ย่ายังดีที่สิริตอบแบบนั้น ดูของ เขาสิ
...,...,...
15571,ไม่ทราบว่าทําการสมัครบริการซื้อสินค้าทางอินเต...,ไม่ทราบว่าทําการสมัครบริการซื้อสินค้าทาง อินเ...
15572,จะสมัคยังไงคัฟ,จะสมัครอย่างไรครับ
15573,อยากรุ้ว่าจะเข้าระบบ ยังไงค่ะ,อยาก รู้ว่าจะเข้าระบบ อย่างไรคะ
15574,เข้าไม่ได้ เปนไรครับ,เข้าไม่ได้ เป็นอะ ไรครับ


In [ ]:
seq_train, seq_val, seq_test = np.split(seq_ds.sample(frac=1, random_state=42),
                            [int(.8 * len(seq_ds)), int(.9 * len(seq_ds))])

In [ ]:
seq_test

,text,labels
15124,สวัดดีค่า,สวัสดีค่ะ
14791,ไปธนาคารแล้วใบสมัครหมด สามารถสมัคร ช่องทางอื่...,ไปธนาคารแล้วใบสมัครหมด สามารถสมัคร ช่องทางอื่...
10395,บัตรเครดิตอ่ะค่ะ,บัตรเครดิตอะค่ะ
2713,ตอนนี้ก้ไม่ได้แชร์สัญญาณค่ะ ใช้อินเตอร์เน็ตขอ...,ตอนนี้ ก็ไม่ได้แชร์สัญญาณค่ะ ใช้อินเตอร์เน็ตข...
4757,แล้วพอดีเปลียนโทรสับใหม่ พอจะเข้าระบบ มันเข้า...,แล้วพอดีเปลี่ยนโทรศัพท์ใหม่ พอจะเข้าระบบ มันเ...
...,...,...
5191,เติมตังโทรศัพกดเลขอะไรค่ะ,เติมสตางค์โทรศัพท์กดเลขอะไรคะ
13418,บัญชีออมทรัพย์นี้ถ้าส่วนใหญ่ฝากอย่างเดียว ไม่...,บัญชีออมทรัพย์นี้ถ้าส่วนใหญ่ฝากอย่างเดียว ไม่...
5390,หากยังไม่สามารถเข้าระบบ ได้ แนะนําให้ลองลบแอพ...,หากยังไม่สามารถเข้าระบบ ได้ แนะนําให้ลองลบแอป...
860,ผ่านอินเตอร์เน็ต,ผ่าน อินเทอร์เน็ต


In [ ]:
def ids_to_tokens(tokenized_text):
  a = tokenizer.convert_ids_to_tokens(tokenized_text)
  a.remove("<s>")
  a.remove("</s>")
  if a[0] == '▁':
    a.pop(0)
  return a

In [ ]:
from torch.nn import factory_kwargs
NUM_SAMPLE = 20
cou = 0
bleu = load_metric("bleu")
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_gs_ori = 0
total_gs_pred = 0

# for index, row in seq_test.iterrows():
#   cou += 1
#   if cou > NUM_SAMPLE:
#     break
#   print(f"({cou})")
for sent_id in tqdm(range(NUM_SAMPLE)):
  row = seq_ds.iloc[sent_id]
  chng = []
  text = row['text']
  # tokenized_text = tokenizer(text, padding='max_length', max_length=512, truncation=True)
  tokenized_text = tokenizer(text)
  mask = tokenized_text['attention_mask']
  tokenized_text = tokenized_text['input_ids']
  labels = row['labels']
  # tokenized_labels = tokenizer(labels, padding='max_length', max_length=512, truncation=True)['input_ids']
  tokenized_labels = tokenizer(labels)['input_ids']
  original = [ids_to_tokens(tokenized_text)]
  references = [[ids_to_tokens(tokenized_labels)]]
  moi = ''.join(original[0])
  moi = moi.replace("▁", " ")
  print(f"TEXT: {moi}")
  
  i_f = evaluate_one_text(tagging_model, text)
  predicted_id = tokenized_text.copy()
  original_id = tokenized_text.copy()
  i_f_len = len(i_f)
  for j in range(i_f_len):
    if(i_f[j] == 'i'):
      ph = predicted_id[j+1]
      predicted_id[j+1] = 25004
      mlm_input = {'input_ids': torch.Tensor([predicted_id]).type(torch.int64).to(device), 'attention_mask': torch.Tensor([mask]).type(torch.int64).to(device)}
      token_logits = mlm_model(**mlm_input).logits
      mask_token_index = torch.where(mlm_input["input_ids"] == tokenizer.mask_token_id)[1]
      mask_token_logits = token_logits[0, mask_token_index, :]
      top_5_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()
      print(f"{tokenizer.convert_ids_to_tokens(ph)} => {tokenizer.convert_ids_to_tokens(top_5_tokens[0])}")
      chng.append((j, top_5_tokens[0]))

      predicted_id[j+1] = ph

  for x,y in chng:
    predicted_id[x+1] = y

  ans = tokenizer.convert_ids_to_tokens(predicted_id)
  ans.remove('<s>')
  ans.remove('</s>')
  if ans[0] == '▁':
    ans.pop(0)
  predictions = [ans]  
  ans = ''.join(ans)
  ans = ans.replace("▁", " ")

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)
  
  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  print(references)
  print(predictions)
  gleu_score = sentence_gleu(references[0], predictions[0], min_len=1, max_len=4)
  total_gs_pred += gleu_score
  gleu_score_original = sentence_gleu(references[0], original[0], min_len=1, max_len=4)
  total_gs_ori += gleu_score_original

  print(f"GLUE PREDICTED: {gleu_score}")
  print(f"GLUE ORIGINAL: {gleu_score_original}")
  # print(ans)
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")
  # print("--------------------------------------------")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")
print(f"# GLEU PREDICTION: {float(total_gs_pred/NUM_SAMPLE)}")
print(f"# GLEU ORIGINAL: {float(total_gs_ori/NUM_SAMPLE)}")

  0%|          | 0/20 [00:00<?, ?it/s]

TEXT: หริอมีปัญหาอะไรช่วยตอบด้วยครับ
ids_to_tokens: ['<s>', '▁', 'ห', 'ริ', 'อ', 'มีปัญหา', 'อะไร', 'ช่วยตอบ', 'ด้วยครับ', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad

  5%|▌         | 1/20 [00:03<00:57,  3.00s/it]

อ => _
[[['หรือ', 'มีปัญหา', 'อะไร', 'ช่วยตอบ', 'ด้วยครับ']]]
[['ห', 'ริ', '_', 'มีปัญหา', 'อะไร', 'ช่วยตอบ', 'ด้วยครับ']]
GLUE PREDICTED: 0.45454545454545453
GLUE ORIGINAL: 0.45454545454545453
TEXT:  อยากสมัคร  sms เงินเข้าเงินออกทางเน็ตต้องทํายังงัย
ids_to_tokens: ['<s>', '▁อยาก', 'สมัคร ', '▁', 'sms', '▁', 'เงิน', 'เข้า', 'เงิน', 'ออก', 'ทาง', 'เน็ต', 'ต้องทํา', 'ยังงัย', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<

 10%|█         | 2/20 [00:05<00:45,  2.54s/it]

...

ม => ม
[[['▁วันนี้', 'เมื่อ', '▁', '4', '▁', 'ปีก่อน', '▁', 'สตรีมมิ่งล่ม', '▁วันนี้', 'พันทิป', 'จะ', 'ล่ม', 'แทน', 'สิ', 'นะครับ']]]
[['▁วันนี้', 'เมื่อ', '▁', '4', '▁', 'ปีก่อน', '▁', 'ตี', 'ม', 'มิ่ง', 'ล่ม', '▁วันนี้', 'พัน', 'ธิ', 'ป', 'จะ', 'ล่ม', 'แทน', 'สิ', 'นะครับ']]
GLUE PREDICTED: 0.5
GLUE ORIGINAL: 0.5
TEXT: รบกวนสอบถามึะ โอนเงินผ่านระบบพร้อมเพ ทําไมยอดไม่เข้า ปลายทาง ทั้งที่ยอดก้ตัดไป แล้ว นานไปไหม
ids_to_tokens: ['<s>', '▁', 'รบกวน', 'สอบถาม', 'ึ', 'ะ', '▁', 'โอนเงิน', 'ผ่านระบบ', 'พร้อม', 'เพ', '▁ทําไม', 'ยอด', 'ไม่เข้า', '▁', 'ป', 'ลายทาง', '▁', 'ทั้งที่', 'ยอด', 'ก้', 'ตัด', 'ไป', '▁แล้ว', '▁', 'นาน', 'ไป', 'ไหม', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 

 95%|█████████▌| 19/20 [00:43<00:02,  2.18s/it]

เพ => แอป
ก้ => ▁ก็
[[['รบกวน', 'สอบถาม', '▁', 'โอนเงิน', 'ผ่านระบบ', 'พร้อม', 'เพ', '▁ทําไม', 'ยอด', 'ไม่เข้า', '▁', 'ป', 'ลายทาง', '▁', 'ทั้งที่', 'ยอด', '▁ก็', 'ตัด', 'ไป', '▁แล้ว', '▁', 'นาน', 'ไป', 'ไหม']]]
[['รบกวน', 'สอบถาม', 'ึ', 'ะ', '▁', 'โอนเงิน', 'ผ่านระบบ', 'พร้อม', 'แอป', '▁ทําไม', 'ยอด', 'ไม่เข้า', '▁', 'ป', 'ลายทาง', '▁', 'ทั้งที่', 'ยอด', '▁ก็', 'ตัด', 'ไป', '▁แล้ว', '▁', 'นาน', 'ไป', 'ไหม']]
GLUE PREDICTED: 0.7653061224489796
GLUE ORIGINAL: 0.7653061224489796
TEXT: แต่งหน้าผีให้น่ากลัวที่สุด เอ็ฟเฟ็คแบบจัดเต็มระดับฮอลิวู้ด แล้วไปหาเพื่อนกลางดึกเคาะประตูเพื่อบอกว่า <unk> appy <unk> aloween
ids_to_tokens: ['<s>', '▁', 'แต่งหน้า', 'ผี', 'ให้', 'น่ากลัว', 'ที่สุด', '▁', 'เอ็', 'ฟ', 'เฟ', '็ค', 'แบบ', 'จัดเต็ม', 'ระดับ', 'ฮอ', 'ลิ', 'วู้ด', '▁แล้ว', 'ไปหา', 'เพื่อน', 'กลางดึก', 'เคาะประตู', 'เพื่อ', 'บอกว่า', '▁', '<unk>', '▁', 'app', 'y', '▁', '<unk>', '▁', 'a', 'lo', 'ween', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',

100%|██████████| 20/20 [00:45<00:00,  2.29s/it]

[[['แต่งหน้า', 'ผี', 'ให้', 'น่ากลัว', 'ที่สุด', '▁', 'เอ็', 'ฟ', 'เฟ', '็ค', 'แบบ', 'จัดเต็ม', 'ระดับ', 'ฮอลลีวูด', '▁แล้ว', 'ไปหา', 'เพื่อน', 'กลางดึก', 'เคาะประตู', 'เพื่อ', 'บอกว่า', '▁', '<unk>', '▁', 'app', 'y', '<unk>', '▁', 'a', 'lo', 'ween']]]
[['แต่งหน้า', 'ผี', 'ให้', 'น่ากลัว', 'ที่สุด', '▁', 'เอ็', 'ฟ', 'เฟ', '็ค', 'แบบ', 'จัดเต็ม', 'ระดับ', 'ฮอ', 'ลิ', 'วู้ด', '▁แล้ว', 'ไปหา', 'เพื่อน', 'กลางดึก', 'เคาะประตู', 'เพื่อ', 'บอกว่า', '▁', '<unk>', '▁', 'app', 'y', '▁', '<unk>', '▁', 'a', 'lo', 'ween']]
GLUE PREDICTED: 0.7846153846153846
GLUE ORIGINAL: 0.7846153846153846
# HIGHER BLEU PREDICTION: 2
# LOWER BLEU PREDICTION: 2
# EQUAL BLEU PREDICTION: 16
# GLEU PREDICTION: 0.5994762800163498
# GLEU ORIGINAL: 0.5513998636790639
